In [2]:
# Import Packages
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
pd.set_option('display.max_colwidth', 100)
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
from gensim.models import KeyedVectors
import gensim.downloader
from collections import deque

In [3]:
# Load pretrained word2vec model from gensim
google_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

In [6]:
# load csv from zip file
main_data = pd.read_csv('/Users/andrewsimon/Desktop/Dow_dat.csv')
main_data

,Level,Report
0,2,"On April 10th at 12:30 PM, there was an incident at the chemical plant. The incident occurred du..."
1,3,"On April 8th at 9:00 PM, a fire broke out in the storage area of the chemical plant. The fire wa..."
2,4,"On April 6th at 2:45 AM, there was an incident at the chemical plant. The incident occurred duri..."
3,1,"On April 5th at 7:15 PM, there was an unplanned event at the chemical plant. The incident occurr..."
4,5,"On April 2nd at 11:30 AM, there was a minor incident at the chemical plant. The incident occurre..."
...,...,...
295,3,"At 9:45 AM, a worker reported a small leak of chemical B in the storage area. The leak was cause..."
296,5,"At 2:30 PM, a worker reported a small spill of chemical C in the separation unit. The spill was ..."
297,4,"At 6:15 AM, a worker reported an unusual odor coming from the storage area. Upon investigation, ..."
298,2,"At 11:00 AM, a worker reported a small fire in the reaction unit. The fire was quickly extinguis..."


In [7]:
# Preprocess data for W2v
main_data['Report'] = main_data['Report'].apply(lambda x: gensim.utils.simple_preprocess(x))
main_data.head()

,Level,Report
0,2,"[on, april, th, at, pm, there, was, an, incident, at, the, chemical, plant, the, incident, occur..."
1,3,"[on, april, th, at, pm, fire, broke, out, in, the, storage, area, of, the, chemical, plant, the,..."
2,4,"[on, april, th, at, am, there, was, an, incident, at, the, chemical, plant, the, incident, occur..."
3,1,"[on, april, th, at, pm, there, was, an, unplanned, event, at, the, chemical, plant, the, inciden..."
4,5,"[on, april, nd, at, am, there, was, minor, incident, at, the, chemical, plant, the, incident, oc..."


In [8]:
corpus = []

for row in range(len(main_data)):
    corpus.append(main_data['Report'][row])

In [15]:
embedded_array = []

for sentence in corpus:
    embedded_sub_array = []
    for word in sentence:
        try: 
            embedded_sub_array.append(list(google_vectors[word]))
        except KeyError:
            embedded_sub_array.append(np.zeros(300,))
            print(2)
    embedded_array.append(embedded_sub_array)

In [32]:

# Append vectorizations from pre trained model

for i in range(len(main_data['review'])):
    sub_list = []
    for j in main_data['review'][i]:
        try:
            sub_list.append(google_vectors[j])
        except KeyError:
            sub_list.append(np.zeros(300,))
    main_data['review'][i] = sub_list


/var/folders/8f/mjj28lvs5fb1m_c_tvmtckl80000gn/T/ipykernel_31124/2852260835.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_data['review'][i] = sub_list


In [29]:
# Separate into training and testing data
X_train, X_test, y_train, y_test = train_test_split(embedded_array, main_data['Level'], test_size=0.15, random_state=1516)

In [30]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'

In [31]:
training_padded = pad_sequences(X_train,maxlen=max_length, truncating=trunc_type)
testing_padded = pad_sequences(X_test,maxlen=max_length, truncating=trunc_type)

In [34]:
# Pad data to remove ragged arrays
training_padded = pad_sequences(X_train, maxlen=120, truncating='post')

In [35]:

testing_padded = pad_sequences(X_test, maxlen=120, truncating='post')

In [36]:
# Cast to numpy arrays
X_train = X_train.to_numpy()

In [37]:
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [28]:
X_train = np.asarray(X_train, dtype=object).astype('float32')
X_test = np.asarray(X_test, dtype=object).astype('float32')
y_train = np.asarray(y_train, dtype = object).astype('float32')
y_test = np.asarray(y_test, dtype = object).astype('float32')

ValueError: setting an array element with a sequence.

In [32]:
# Create model, evaluate

batch = 32

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy', 'Recall', 'AUC', 'Precision', 'FalseNegatives', 'FalsePositives'])
    
num_epochs = 10
model.fit(training_padded, y_train, epochs=num_epochs,batch_size=batch, validation_data=(testing_padded, y_test))

Epoch 1/10
8/8 [==============================] - 1s 27ms/step - loss: -3.0747 - accuracy: 0.1412 - recall: 0.9647 - auc: 0.0000e+00 - precision: 1.0000 - false_negatives: 9.0000 - false_positives: 0.0000e+00 - val_loss: -7.9215 - val_accuracy: 0.2000 - val_recall: 1.0000 - val_auc: 0.0000e+00 - val_precision: 1.0000 - val_false_negatives: 0.0000e+00 - val_false_positives: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 0s 6ms/step - loss: -11.2233 - accuracy: 0.1529 - recall: 1.0000 - auc: 0.0000e+00 - precision: 1.0000 - false_negatives: 0.0000e+00 - false_positives: 0.0000e+00 - val_loss: -16.2763 - val_accuracy: 0.2000 - val_recall: 1.0000 - val_auc: 0.0000e+00 - val_precision: 1.0000 - val_false_negatives: 0.0000e+00 - val_false_positives: 0.0000e+00
Epoch 3/10
8/8 [==============================] - 0s 6ms/step - loss: -19.4794 - accuracy: 0.1529 - recall: 1.0000 - auc: 0.0000e+00 - precision: 1.0000 - false_negatives: 0.0000e+00 - false_positives: 0.0000e+00 - val_lo

In [40]:
training_padded_w2v = training_padded
testing_padded_w2v = testing_padded

%store training_padded_w2v
%store testing_padded_w2v

Stored 'training_padded_w2v' (ndarray)
Stored 'testing_padded_w2v' (ndarray)
